Launch interactive version: 👉👉👉 [![Try ``dyce``](https://jupyterlite.readthedocs.io/en/latest/_static/badge.svg)](https://posita.github.io/dyce-notebooks/lab?path=stack-exchange%2Fironsworn-in-the-dark-200325%2Fironsworn_in_the_dark.ipynb) 👈👈👈 *[[source](https://github.com/posita/dyce-notebooks/tree/main/notebooks/stack-exchange/ironsworn-in-the-dark-200325)]*

## [``dyce``](https://posita.github.io/dyce/) solution to [“Ironsworn meets BitD dice mechanic (with a twist)”](https://rpg.stackexchange.com/a/200357/71245)

Once viewing this notebook in Jupyter Lab, select ``Run All Cells`` from the ``Run`` menu above.

In [1]:
# Install additional requirements if necessary
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    try:
        import anydyce
    except (ImportError, ModuleNotFoundError):
        # See <https://jupyterlite.readthedocs.io/en/stable/howto/configure/simple_extensions.html#avoid-the-drift-of-versions-between-the-frontend-extension-and-the-python-package>
        requirements = ["ipywidgets==8.1.3", "matplotlib==3.5.2", "anydyce==0.4.6"]
        try:
            import piplite ; await piplite.install(requirements, keep_going=True)
            # Work around <https://github.com/jupyterlite/jupyterlite/issues/838>
            import matplotlib.pyplot ; matplotlib.pyplot.clf()
        except ImportError:
            import pip ; pip.main(["install"] + requirements)
    import anydyce

In [2]:
from dyce import P
from dyce.evaluation import PResult, foreach

def ironsworn_bitd(action: PResult, challenge: PResult) -> int:
    max_action = max(action.roll, default=0)
    max_challenge = max(challenge.roll, default=0)
    if max_action <= 0 and max_challenge <= 0:
        return 0
    basic_successes = sum(1 for c in challenge.roll if c < max_action)
    num_action_matches = sum(1 for a in action.roll if a == max_action) - 1
    if max_action < max_challenge:
        match_successes = 0
    elif max_action > max_challenge:
        match_successes = num_action_matches
    else:  # max_challenge == max_action:
        num_challenge_matches = sum(1 for c in challenge.roll if c == max_action) - 1
        # Offset the number of action matches by the number of challenge matches up to
        # the number of action matches
        match_successes = max(0, num_action_matches - num_challenge_matches)
    return basic_successes + match_successes

h = foreach(ironsworn_bitd, action=4@P(6), challenge=2@P(6))
print(h.format(scaled=True))

avg |    1.66
std |    0.92
var |    0.85
  0 |  10.95% |############
  1 |  29.67% |#################################
  2 |  43.74% |##################################################
  3 |  13.53% |###############
  4 |   1.99% |##
  5 |   0.12% |


In [3]:
from anydyce import jupyter_visualize

jupyter_visualize(
    (
        (f"{a}d6 action vs. 2d6 challenge", foreach(ironsworn_bitd, action=a @ P(6), challenge=2 @ P(6)))
        for a in range(1, 6)
    ),
    initial_burst_zero_fill_normalize=True,
)